## only pose

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Step 1: Ask for a name
name = input("Enter your name: ")

# Step 2: Create folder
folder_path = os.path.join(os.getcwd(), name)
os.makedirs(folder_path, exist_ok=True)
print(f"Saving frames to: {folder_path}")

# Step 3: Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Step 4: Start Pose with segmentation
with mp_pose.Pose(static_image_mode=False,
                  model_complexity=1,
                  enable_segmentation=True,
                  min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    frame_count = 0  # To save frames with unique names

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Pose estimation
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Create white background
        bg_color = (255, 255, 255)
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = bg_color

        if results.segmentation_mask is not None:
            condition = results.segmentation_mask > 0.02
            output_image = np.where(condition[..., None], image, bg_image)
        else:
            output_image = image

        # Draw landmarks if present
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                output_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Step 5: Save each frame
        filename = os.path.join(folder_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(filename, output_image)
        frame_count += 1

        # Show preview
        cv2.imshow('Pose with Background Removed', output_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Enter your name:  ank


Saving frames to: E:\removeBackground\ank


## hand + pose

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Step 1: Ask for a name
name = input("Enter your name: ")

# Step 2: Create folder
folder_path = os.path.join(os.getcwd(), name)
os.makedirs(folder_path, exist_ok=True)
print(f"Saving frames to: {folder_path}")

# Initialize MediaPipe modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

# Use both Pose and Hands
with mp_pose.Pose(static_image_mode=False,
                  model_complexity=1,
                  enable_segmentation=True,
                  min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose, \
     mp_hands.Hands(static_image_mode=False,
                    max_num_hands=2,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as hands:

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False

        # Step 1: Pose estimation
        pose_results = pose.process(image_rgb)

        # Step 2: Hand detection
        hand_results = hands.process(image_rgb)

        image_rgb.flags.writeable = True
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Create white background
        bg_image = np.ones(image_bgr.shape, dtype=np.uint8) * 255

        # Use pose segmentation mask for background removal
        if pose_results.segmentation_mask is not None:
            condition = pose_results.segmentation_mask > 0.02
            output_image = np.where(condition[..., None], image_bgr, bg_image)
        else:
            output_image = image_bgr

        # Draw pose landmarks
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                output_image,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS)

        # Draw hand landmarks
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    output_image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS)

        # Save frame to folder
        filename = os.path.join(folder_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(filename, output_image)
        frame_count += 1

        # Show preview
        cv2.imshow('Pose + Hands with Background Removed', output_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Enter your name:  ankit


Saving frames to: E:\removeBackground\ankit


TypeError: __init__() got an unexpected keyword argument 'enable_segmentation'

## no bg remove and save

In [13]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Step 1: Ask for a name
name = input("Enter your name: ")

# Step 2: Create folder
folder_path = os.path.join(os.getcwd(), name)
os.makedirs(folder_path, exist_ok=True)
print(f"Saving frames to: {folder_path}")

# Initialize MediaPipe modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

# Use both Pose and Hands (NO segmentation needed)
with mp_pose.Pose(static_image_mode=False,
                  model_complexity=1,
                  enable_segmentation=False,  # ✅ Segmentation turned off
                  min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose, \
     mp_hands.Hands(static_image_mode=False,
                    max_num_hands=2,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as hands:

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False

        # Step 1: Pose estimation
        pose_results = pose.process(image_rgb)

        # Step 2: Hand detection
        hand_results = hands.process(image_rgb)

        image_rgb.flags.writeable = True
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # We skip background removal and use original image directly
        output_image = image_bgr.copy()

        # Draw pose landmarks
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                output_image,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS)

        # Draw hand landmarks
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    output_image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS)

        # Save frame to folder
        filename = os.path.join(folder_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(filename, output_image)
        frame_count += 1

        # Show preview
        cv2.imshow('Pose + Hands (No Background Removed)', output_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Enter your name:  hello


Saving frames to: E:\removeBackground\hello


In [11]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Ask for the folder path
folder_path = input("Enter the full path of the folder with images: ").strip()

# Check if folder exists
if not os.path.isdir(folder_path):
    print("Folder does not exist. Exiting.")
    exit()

# Initialize MediaPipe Pose with segmentation
mp_pose = mp.solutions.pose

with mp_pose.Pose(static_image_mode=True,  # Use static mode for images
                  model_complexity=1,
                  enable_segmentation=True,
                  min_detection_confidence=0.5) as pose:

    # Process each image file
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing: {file_path}")

            # Read image
            image = cv2.imread(file_path)
            if image is None:
                print(f"Failed to read {file_path}. Skipping.")
                continue

            # Convert to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            # Create white background
            bg_image = np.ones(image.shape, dtype=np.uint8) * 255

            # If segmentation mask is available
            if results.segmentation_mask is not None:
                condition = results.segmentation_mask > 0.07
                output_image = np.where(condition[..., None], image, bg_image)
            else:
                output_image = image

            # Overwrite the original image
            cv2.imwrite(file_path, output_image)

    print("All images processed.")


Enter the full path of the folder with images:  ag


Processing: ag\frame_0000.jpg
Processing: ag\frame_0001.jpg
Processing: ag\frame_0002.jpg
Processing: ag\frame_0003.jpg
Processing: ag\frame_0004.jpg
Processing: ag\frame_0005.jpg
Processing: ag\frame_0006.jpg
Processing: ag\frame_0007.jpg
Processing: ag\frame_0008.jpg
Processing: ag\frame_0009.jpg
Processing: ag\frame_0010.jpg
Processing: ag\frame_0011.jpg
Processing: ag\frame_0012.jpg
Processing: ag\frame_0013.jpg
Processing: ag\frame_0014.jpg
Processing: ag\frame_0015.jpg
Processing: ag\frame_0016.jpg
Processing: ag\frame_0017.jpg
Processing: ag\frame_0018.jpg
Processing: ag\frame_0019.jpg
Processing: ag\frame_0020.jpg
Processing: ag\frame_0021.jpg
Processing: ag\frame_0022.jpg
Processing: ag\frame_0023.jpg
Processing: ag\frame_0024.jpg
Processing: ag\frame_0025.jpg
Processing: ag\frame_0026.jpg
Processing: ag\frame_0027.jpg
Processing: ag\frame_0028.jpg
Processing: ag\frame_0029.jpg
Processing: ag\frame_0030.jpg
Processing: ag\frame_0031.jpg
Processing: ag\frame_0032.jpg
Processing

In [1]:
# !pip install pillo


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
